# Homework 1

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple
import itertools


## CSP utilities

# A CSPVariable has a str name and a set of domain values (ints).
CSPVariable = namedtuple("CSPVariable", ["name", "domain"])
# Makes CSPVariables hashable.
CSPVariable.__hash__ = lambda self: hash((self.name, frozenset(self.domain)))


class BinaryConstraint:
  """A constraint on two CSPVariables.

  See child classes.
  """
  def __init__(self, var1, var2):
    self.var1 = var1
    self.var2 = var2

  def check(self, var1_value, var2_value):
    """Checks whether the constraint holds for an assignment.

    Args:
      var1_value: A value in the domain of self.var1.
      var2_value: A value in the domain of self.var2.

    Returns:
      holds: True if the constraint holds.
    """
    raise NotImplementedError("Implemented by child classes.")


class ImplicitBinaryConstraint(BinaryConstraint):
  """A binary constraint defined by a lambda function.

  Example usage:
    var1 = CSPVariable("A", {0, 1})
    var2 = CSPVariable("B", {0, 1})
    fn = lambda a, b: (a == b)
    constraint = ImplicitBinaryConstraint(var1, var2, fn)
    constraint.check(0, 0)
  """
  def __init__(self, var1, var2, constraint_fn):
    """Initializes an ImplicitBinaryConstraint.

    Args:
      var1: A CSPVariable.
      var2: A CSPVariable.
      constraint_fn: A callable that takes a value for var1 and
        a value for var2 and returns a bool.
    """
    super().__init__(var1, var2)
    self._constraint_fn =  constraint_fn

  def check(self, var1_value, var2_value):
    """Checks whether the constraint holds for an assignment.

    Args:
      var1_value: A value in the domain of self.var1.
      var2_value: A value in the domain of self.var2.

    Returns:
      holds: True if the constraint holds.
    """
    assert var1_value in self.var1.domain
    assert var2_value in self.var2.domain
    return self._constraint_fn(var1_value, var2_value)

  def __reversed__(self):
    """Return an ImplicitBinaryConstraint with var1 and var2 swapped.

    This method is useful for AC3. You should not need to worry
    about it otherwise.

    Returns:
      new_constraint: A new ImplicitBinaryConstraint.
    """
    new_fn = lambda v2, v1: self._constraint_fn(v1, v2)
    return ImplicitBinaryConstraint(self.var2, self.var1, new_fn)


class ExplicitBinaryConstraint(BinaryConstraint):
  """A binary constraint defined by a set of valid assignment tuples.

  Example usage:
    var1 = CSPVariable("A", {0, 1})
    var2 = CSPVariable("B", {0, 1})
    valid_assignments = {(0, 0), (1, 1)}
    constraint = ExplicitBinaryConstraint(var1, var2, valid_assignments)
    constraint.check(0, 0)
  """
  def __init__(self, var1, var2, valid_assignments):
    """Initializes an ExplicitBinaryConstraint.

    Args:
      var1: A CSPVariable.
      var2: A CSPVariable.
      valid_assignments: A set of binary tuples of values for
        var1 and var2 that satisfy the constraint.
    """
    super().__init__(var1, var2)
    assert all(isinstance(a, tuple) and len(a) == 2
               for a in valid_assignments)
    self._valid_assignments =  valid_assignments

  def __repr__(self):
    return f"ExplicitBinaryConstraint({self.var1}, {self.var2}, {self._valid_assignments})"
  
  def check(self, var1_value, var2_value):
    """Checks whether the constraint holds for an assignment.

    Args:
      var1_value: A value in the domain of self.var1.
      var2_value: A value in the domain of self.var2.

    Returns:
      holds: True if the constraint holds.
    """
    assert var1_value in self.var1.domain
    assert var2_value in self.var2.domain
    return (var1_value, var2_value) in self._valid_assignments

  def __reversed__(self):
    """Return an ExplicitBinaryConstraint with var1 and var2 swapped.

    This method is useful for AC3. You should not need to worry
    about it otherwise.

    Returns:
      new_constraint: A new ExplicitBinaryConstraint.
    """
    new_assigns = {(val2, val1) for (val1, val2) in self._valid_assignments}
    return ExplicitBinaryConstraint(self.var2, self.var1, new_assigns)


# A BinaryCSP is defined by a set of CSPVariables and a set of Constraints
BinaryCSP = namedtuple("BinaryCSP", ["variables", "constraints"])


class VariableToPossibleValuesMap:
  """Stores a set of possible values for each variable in a CSP.

  A helpful data structure for solving CSPs.

  Example usage:
    var1 = CSPVariable("A", {0, 1})
    var2 = CSPVariable("B", {0, 1})
    var3 = CSPVariable("C", {0, 1, 2})
    # Initializes each variable's possible values to its full domain
    vars_to_values = VariableToPossibleValuesMap({
      var1: var1.domain,
      var2: var2.domain,
      var3: var3.domain,
    })
    assert vars_to_values.get(var1) == {0, 1}
    assert vars_to_values.get(var2) == {0, 1}
    assert vars_to_values.get(var3) == {0, 1, 2}
    # Creates a new VariableToPossibleValuesMap with the values reassigned
    new_vars_to_values = vars_to_values.assign(var1, {0})
    assert new_vars_to_values.get(var1) == {0}
    assert new_vars_to_values.get(var2) == {0, 1}
    # Original vars_to_values is unchanged
    assert vars_to_values.get(var1) == {0, 1}
  """
  def __init__(self, vars_to_values):
    """Initializes a VariableToPossibleValuesMap.

    Args:
      vars_to_values: A dict from CSPVariables to sets of values.
    """
    self._vars_to_values = vars_to_values

  def __repr__(self):
    return f"VariableToPossibleValuesMap({self._vars_to_values})"

  def get(self, variable):
    """Get the possible values for the given variable.

    Args:
      variable: A CSPVariable in this map.

    Returns:
      values: A set of values in this map for the given variable.
    """
    return set(self._vars_to_values[variable])

  def assign(self, variable, new_values):
    """Creates a new VariableToPossibleValuesMap with the variable's
        possible values reassigned to new_values.

    Args:
      variable: A CSPVariable in this map.
      new_values: A set of new values, a subset of variable's domain.

    Returns:
      new_map: A new VariableToPossibleValuesMap.
    """
    assert set(new_values).issubset(variable.domain)
    new_vars_to_values = {v: set(vals)
        for v, vals in self._vars_to_values.items()}
    new_vars_to_values[variable] = set(new_values)
    new_map = VariableToPossibleValuesMap(new_vars_to_values)
    return new_map

  def remove(self, variable, value):
    """Creates a new VariableToPossibleValuesMap with the variable's
        possible values excluding value.
    
    Args:
      variable: A CSPVariable in this map.
      value: A value in variable's domain.

    Returns:
      new_map: A new VariableToPossibleValuesMap.
    """
    assert value in variable.domain
    assert value in self._vars_to_values[variable], \
        "Tried to remove a value that was not in the map."
    new_vars_to_values = {v: set(vals)
        for v, vals in self._vars_to_values.items()}
    new_vars_to_values[variable].remove(value)
    new_map = VariableToPossibleValuesMap(new_vars_to_values)
    return new_map

def reduce_csp_ac3(csp, vars_to_values):
  """Use AC-3 to reduce the possible values for variables in a binary CSP.

  Args:
    csp: A BinaryCSP.
    vars_to_values: A VariableToPossibleValuesMap.

  Returns:
    new_vars_to_values : A new VariableToPossibleValuesMap,
      or None, if an inconsistency is found.

  Example:
    x1 = CSPVariable("x1", [1, 2, 3])
    x2 = CSPVariable("x2", [1, 2, 3])
    x3 = CSPVariable("x3", [1, 2, 3])
    vars = {x1, x2, x3}
    constraints = [
        ExplicitBinaryConstraint(x1, x2, {(1, 1)}),
        ImplicitBinaryConstraint(x1, x3, lambda x1, x3: x1 == x3),
    ]
    csp = BinaryCSP(vars, constraints)
    vars_to_values = VariableToPossibleValuesMap({
      x1: [1, 2, 3],
      x2: [1, 2, 3],
      x3: [1, 2, 3],
    })
    new_vars_to_values = reduce_csp_ac3(csp, vars_to_values)
    assert new_vars_to_values.get(x1) == {1}
    assert new_vars_to_values.get(x2) == {1}
    assert new_vars_to_values.get(x3) == {1}
  """
  # Initialize the queue
  queue = []
  for constraint in csp.constraints:
    forward_arc = (constraint.var1, constraint.var2, constraint)
    reverse_arc = (constraint.var2, constraint.var1, reversed(constraint))
    queue.extend([forward_arc, reverse_arc])
  while queue:
    xi, xj, constraint = queue.pop()
    # Revise
    revised = False
    for x in vars_to_values.get(xi):
      y_exists = False
      for y in vars_to_values.get(xj):
        if constraint.check(x, y):
          y_exists = True
          break
      # No value of xj can satisfy the constraint under Xi=x
      if not y_exists:
        # Delete x from xi
        vars_to_values = vars_to_values.remove(xi, x)
        revised = True
    if revised:
      # If there are no possible values for xi, inconsistency
      if not vars_to_values.get(xi):
        return None
      for neighbor_arc in get_binary_csp_neighbor_arcs(xi, csp.constraints):
        if xj in neighbor_arc:
          continue
        queue.append(neighbor_arc)
  return vars_to_values


def get_binary_csp_neighbor_arcs(variable, constraints):
  """Helper for ac3.
  """
  for constraint in constraints:
    if variable == constraint.var1 or variable == constraint.var2:
      yield (constraint.var1, constraint.var2, constraint)
      yield (constraint.var2, constraint.var1, reversed(constraint))


def value_is_consistent(variable, value, constraints, partial_assignment):
  """Determine if assigning variable value would violate any constraints.

  Args:
    variable: A CSPVariable.
    value: A value in the variable's domain.
    constraints: A set of BinaryConstraints.
    partial_assignment: A dict from CSPVariables to values.

  Returns:
    consistent: A bool that is false if there is a violation.
  """
  for constraint in constraints:
    if constraint.var1 == variable and constraint.var2 in partial_assignment:
      if not constraint.check(value, partial_assignment[constraint.var2]):
        return False
    if constraint.var2 == variable and constraint.var1 in partial_assignment:
      if not constraint.check(partial_assignment[constraint.var1], value):
        return False
  return True


def select_unassigned_variable(csp, vars_to_values, partial_assignment):
  """Use the minimum-remaining-values heuristic to select a variable.

  Args:
    csp: A BinaryCSP.
    vars_to_values: A VariableToPossibleValuesMap.
    partial_assignment: A dict from variables to values in their domain.

  Returns:
    variable: A str variable name.
  """
  unassigned_variables = [v for v in csp.variables if v not in partial_assignment]
  return min(unassigned_variables, key=lambda v: len(vars_to_values.get(v)))


def count_induced_constraints(variable, value, csp, vars_to_values):
  """Helper for order_variable_values (least-constraining-value).
  """
  num_possibilities_before = sum(len(vars_to_values.get(v)) for v in csp.variables)
  num_possibilities_after = 0
  for var in csp.variables:
    for var_value in vars_to_values.get(var):
      if value_is_consistent(var, var_value, csp.constraints, {variable: value}):
        num_possibilities_after += 1
  num_induced_constraints = num_possibilities_before - num_possibilities_after
  assert num_possibilities_after <= num_possibilities_before
  return num_induced_constraints


def order_variable_values(variable, csp, vars_to_values):
  """Order a variable's remaining values using the least-constraining value heuristic.

  Args:
    variable: The CSPVariable that we want to order.
    csp: A BinaryCSP.
    vars_to_values: A VariableToPossibleValuesMap.

  Returns:
    values: The domain values sorted.
  """
  fn = lambda v: count_induced_constraints(variable, v, csp, vars_to_values)
  return sorted(vars_to_values.get(variable), key=fn)

## Propositional logic utility functions

def is_cnf_formula(formula):
  """Checks whether the input is a valid CNF formula.

  A formula is in valid CNF form if it is a list of lists of nonzero
  integers with sign indicating whether the variable is negated.

  You will not need to use this utility in your implementation,
  but it may be useful to read to understand the CNF representation.
  """
  if not isinstance(formula, list):
    return False
  if len(formula) == 0:
    return True
  clause = formula[0]
  if not isinstance(clause, list):
    return False
  for literal in clause:
    if not isinstance(literal, int):
      return False
    if literal == 0:
      return False
  if len(formula) == 1:
    return True
  return is_cnf_formula(formula[1:])


def get_variables_in_cnf_formula(cnf_formula):
  """Get a list of all variables in a CNF formula.

  Args:
    cnf_formula: A list of lists of nonzero ints.

  Returns:
    variables: A list of all variables that appear in
      the formula.
  """
  variables = set()
  for clause in cnf_formula:
    variables.update({lit_to_var_val(literal)[0] for literal in clause})
  variables = sorted(variables)
  return variables


def clause_is_determined(clause, partial_assignment):
  """Checks whether all variables in the clause have an assignment.

  Args:
    clause: A list of nonzero ints.
    partial_assignment: A dict of variables (ints) to values (bools).

  Returns:
    is_determined: True if all variables in the clause appear in
      partial_assignment.
  """
  for literal in clause:
    if not (literal in partial_assignment or -literal in partial_assignment):
      return False
  return True


def lit_to_var_val(literal):
  """Converts a literal into (variable, value).

  Args:
    literal: A nonzero int.

  Returns:
    variable: A positive int representing a variable.
    value: True or False, i.e., positive or negative.
  """
  return abs(literal), literal > 0


def literal_is_satisfied(literal, partial_assignment):
  """Checks whether the literal is satisfied by the assignment.

  Args:
    literal: A nonzero int.
    partial_assignment: A dict of variables (ints) to values (bools).

  Returns:
    is_satisfied: True if the literal's variable appears in the
      partial_assignment, with a sign matching the literal.
  """
  variable, val = lit_to_var_val(literal)
  return variable in partial_assignment and partial_assignment[variable] == val


def clause_is_satisfied(clause, partial_assignment):
  """Checks whether the clause is satisfied by the assignment.

  Args:
    clause: A list of nonzero ints.
    partial_assignment: A dict of variables (ints) to values (bools).

  Returns:
    is_satisfied: True if all the literals in the clause are
      satisfied.      
  """
  for literal in clause:
    if literal_is_satisfied(literal, partial_assignment):
      return True
  return False


def find_pure_variable(cnf_formula, variables, partial_assignment):
  """Helper for DPLL.
  
  A variable is pure if it has the same sign in all unsatisfied clauses
  and if it is not already assigned.

  If a pure variable exists, this function returns the variable and value
  corresponding to the literal. (If multiple exist, return an arbitrary one.)

  If no pure variables exist, return (None, None).

  Args:
    cnf_formula: A list of lists of nonzero integers representing a CNF formula,
      with sign indicating whether the variable is negated.
    variables: A list of positive integers.
    partial_assignment: A dict mapping positive integers to bools, or None for
      an empty assignment.

  Returns:
    variable : A positive integer or None.
    value: A bool or None.
  """
  candidate_to_possible_values  = {v : {True, False} for v in variables \
                                  if v not in partial_assignment}
  for clause in cnf_formula:
    if clause_is_satisfied(clause, partial_assignment):
      continue
    for literal in clause:
      variable, value = lit_to_var_val(literal)
      if variable in candidate_to_possible_values:
        candidate_to_possible_values[variable].discard(not value)
  for candidate, possible_values in candidate_to_possible_values.items():
    if possible_values:
      value = next(iter(possible_values))
      return candidate, value
  return None, None


def find_unit_clause(cnf_formula, partial_assignment):
  """Helper for DPLL.
  
  A clause is a unit clause if all literals but one are already assigned to
  False. If a unit clause exists, this function returns the variable and value
  corresponding to the literal. (If multiple exist, return an arbitrary one.)

  Args:
    cnf_formula: A list of lists of nonzero integers representing a CNF formula,
      with sign indicating whether the variable is negated.
    variables: A list of positive integers.
    partial_assignment: A dict mapping positive integers to bools, or None for
      an empty assignment.

  Returns:
    variable : A positive integer or None.
    value: A bool or None.
  """
  for clause in cnf_formula:
    unassigned_literal = None
    is_unit_clause = True
    for literal in clause:
      # If the literal is true in the assignment, this is not a unit clause
      if literal_is_satisfied(literal, partial_assignment):
        is_unit_clause = False
        break
      # If the literal is false in the assignment, this could be a unit clause
      elif literal in partial_assignment:
        continue
      # If there is already an unassigned literal, this is not a unit clause
      elif not (unassigned_literal is None):
        is_unit_clause = False
        break
      else:
        unassigned_literal = literal
    if is_unit_clause and not (unassigned_literal is None):
      return lit_to_var_val(unassigned_literal)
  return None, None



## Problems

### CSP Warmup
Create a BinaryCSP with the criteria described in the docstring.

For reference, our solution is **4** lines of code.

In [ ]:
def warmup_create_csp():
  """Creates a BinaryCSP with the following criteria:

    * There are two variables named "x" and "y"
    * Variable "x" has domain {1, 2, 3, 4, 5}
    * Variable "y" has domain {4, 5, 6, 7}
    * There is one constraint: x and y are either both
      even or both odd.

  Returns:
    csp: The BinaryCSP.
  """
  var1 = CSPVariable("x", {1,2,3,4,5})
  var2 = CSPVariable("y", {4,5,6,7})
  c1 = ImplicitBinaryConstraint(var1, var2, lambda var1, var2: (var1+var2)%2 == 0)
  return BinaryCSP({var1,var2},{c1})

Tests

In [ ]:
csp = warmup_create_csp()
assert len(csp.variables) == 2
for v in csp.variables:
  if v.name == "x":
    assert v.domain == {1, 2, 3, 4, 5}
    x = v
  elif v.name == "y":
    assert v.domain == {4, 5, 6, 7}
    y = v
assert len(csp.constraints) == 1
c = next(iter(csp.constraints))
for v1, v2 in {(1, 5), (1, 7), (3, 5), (2, 4), (4, 6), (5, 5)}:
  assert c.check(v1, v2)
for v1, v2 in {(1, 4), (5, 4), (4, 5), (3, 6), (1, 6), (2, 7)}:
  assert not c.check(v1, v2)

print('Tests passed.')

Tests passed.


### Backtracking Search
Complete this implementation of backtracking search for binary CSPs.
- Use the provided implementation of AC-3.
- Use the least-constraining-value heuristic.
- Use the MRV heuristic.

For reference, our solution is **31** lines of code.

In [ ]:
def revise(x, y,csp):
    revised = False
    # Get x and y domains
    x_domain = x.domain
    y_domain = y.domain
    all_constraints = [
        constraint for constraint in csp.constraints if constraint.var1 == x and constraint.var2 == y]
    remove = []
    for x_value in x_domain:
        satisfies = False
        for y_value in y_domain:
            for constraint in all_constraints:
                constraint_func = constraint._constraint_fn
                if constraint_func(x_value, y_value):
                    satisfies = True
        if not satisfies:
            remove.append(x_value)
            revised = True
    for v in remove:
      x_domain.remove(v)
    return revised

def reduce_csp_ac3(csp):
  queue = list(csp.constraints)

    # Repeat until the queue is empty
  while queue:
        # Take the first arc off the queue (dequeue)
      temp = queue.pop(0)
      x = temp.var1
      y = temp.var2

        # Make x arc consistent with y
      revised = revise(x, y,csp)

        # If the x domain has changed
      if revised:
            # Add all arcs of the form (k, x) to the queue (enqueue)
          neighbors = [neighbor for neighbor in csp.constraints if neighbor.var1 == x]
          queue = queue + neighbors
def run_backtracking_search(csp):
  """Solve a binary CSP with backtracking search.

  Uses select_unassigned_variable, order_variable_values, and reduce_csp_ac3.

  Args:
    csp: A BinaryCSP.

  Returns:
    solution : A dict mapping variables to values, or None if no
      solution exists.
  """
  x1 = CSPVariable("x1", {1, 2, 3})
  x2 = CSPVariable("x2", {1, 2, 3})
  x3 = CSPVariable("x3", {1, 2, 3})
  c1 = list(csp.constraints)
  print(c1)
  

Tests

In [ ]:
x1 = CSPVariable("x1", {1, 2, 3})
x2 = CSPVariable("x2", {1, 2, 3})
x3 = CSPVariable("x3", {1, 2, 3})
c1 = ImplicitBinaryConstraint(x1, x2, lambda v1, v2: v1 == v2)
c2 = ImplicitBinaryConstraint(x1, x3, lambda v1, v3: v1 == v3)
c3 = ImplicitBinaryConstraint(x2, x3, lambda v2, v3: v2 != v3)
csp = BinaryCSP({x1, x2, x3}, {c1, c2, c3})
assert run_backtracking_search(csp) == None

x1 = CSPVariable("x1", {1, 2, 3})
x2 = CSPVariable("x2", {1, 2, 3})
x3 = CSPVariable("x3", {1, 2, 3})
c1 = ImplicitBinaryConstraint(x1, x2, lambda v1, v2: v1 + v2 == 3)
c2 = ImplicitBinaryConstraint(x1, x3, lambda v1, v3: v1 + v3 == 4)
c3 = ImplicitBinaryConstraint(x2, x3, lambda v2, v3: v2 + v3 == 5)
csp = BinaryCSP({x1, x2, x3}, {c1, c2, c3})
assert run_backtracking_search(csp) == {x1: 1, x2: 2, x3: 3}

x1 = CSPVariable("x1", {1, 2, 3})
x2 = CSPVariable("x2", {1, 2, 3})
x3 = CSPVariable("x3", {1, 2, 3})
c1 = ImplicitBinaryConstraint(x1, x2, lambda v1, v2: v1 + v2 == 5)
c2 = ImplicitBinaryConstraint(x1, x3, lambda v1, v3: v1 + v3 == 4)
c3 = ImplicitBinaryConstraint(x2, x3, lambda v2, v3: v2 + v3 == 3)
csp = BinaryCSP({x1, x2, x3}, {c1, c2, c3})
assert run_backtracking_search(csp) == {x1: 3, x2: 2, x3: 1}

print('Tests passed.')

[<__main__.ImplicitBinaryConstraint object at 0x7f54cc84fd10>, <__main__.ImplicitBinaryConstraint object at 0x7f54cc84fc50>, <__main__.ImplicitBinaryConstraint object at 0x7f54cc84f950>]


AttributeError: ignored

### Sudoku Solving
Use your implementation of backtracking search to solve sudoku puzzles like the following:
```
sudoku_puzzle = [
  [0, 0, 3, 0, 2, 0, 6, 0, 0],
  [9, 0, 0, 3, 0, 5, 0, 0, 1],
  [0, 0, 1, 8, 0, 6, 4, 0, 0],
  [0, 0, 8, 1, 0, 2, 9, 0, 0],
  [7, 0, 0, 0, 0, 0, 0, 0, 8],
  [0, 0, 6, 7, 0, 8, 2, 0, 0],
  [0, 0, 2, 6, 0, 9, 5, 0, 0],
  [8, 0, 0, 2, 0, 3, 0, 0, 9],
  [0, 0, 5, 0, 1, 0, 3, 0, 0],
]
```

For reference, our solution is **61** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_backtracking_search`. You may not need to use all of them.

In [ ]:
def solve_sudoku_puzzle(puzzle):
  """Fill in all zeros in a partially filled sudoku puzzle.

  Args:
    puzzle: A list of lists of ints. Zero means empty.

  Returns:
    finished_puzzle: A list of lists of ints with no empties.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
assert solve_sudoku_puzzle([[0, 0, 3, 0, 2, 0, 6, 0, 0], 
                            [9, 0, 0, 3, 0, 5, 0, 0, 1], [0, 0, 1, 8, 0, 6, 4, 0, 0], [0, 0, 8, 1, 0, 2, 9, 0, 0], [7, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 6, 7, 0, 8, 2, 0, 0], 
                            [0, 0, 2, 6, 0, 9, 5, 0, 0], [8, 0, 0, 2, 0, 3, 0, 0, 9], [0, 0, 5, 0, 1, 0, 3, 0, 0]]) == [[4, 8, 3, 9, 2, 1, 6, 5, 7], [9, 6, 7, 3, 4, 5, 8, 2, 1], 
                                                                                              [2, 5, 1, 8, 7, 6, 4, 9, 3], [5, 4, 8, 1, 3, 2, 9, 7, 6], [7, 2, 9, 5, 6, 4, 1, 3, 8], 
                                                                                              [1, 3, 6, 7, 9, 8, 2, 4, 5], [3, 7, 2, 6, 8, 9, 5, 1, 4], [8, 1, 4, 2, 5, 3, 7, 6, 9], 
                                                                                              [6, 9, 5, 4, 1, 7, 3, 8, 2]]
print('Tests passed.')

### Warmup
In this problem set, CNF formulas are represented as lists of lists of nonzero integers. The sign of the integer represents whether the corresponding proposition is negated or not. For example, the formula ((x1 or not x2) and (x3 or x2)) would be represented as [[1, -2], [3, 2]]. Complete the following function to confirm your understanding of this representation.

For reference, our solution is **1** lines of code.

In [ ]:
def warmup():
  """Return a list of lists of ints for the CNF formula:
  ((x4 or not x5 or not x6) and (x6 or x5 or not x1) and (x2 or x3)).
  """
  return [[4,-5,-6],[6,5,-1],[2,3]]

Tests

In [ ]:
# Note: the catsoop test is more strict for this question
# than the unit tests here. Make sure that your answer
# matches the description in the docstring exactly.
assert is_cnf_formula(warmup())
assert get_variables_in_cnf_formula(warmup()) == [1, 2, 3, 4, 5, 6]

print('Tests passed.')

Tests passed.


### DPLL
Use your helper functions to complete an implementation of DPLL.

For reference, our solution is **56** lines of code.

In [ ]:
def run_inference_dpll(cnf_formula):
  """Find a satisfying assignment for a propositional CNF formula with DPLL.

  Args:
    cnf_formula: A list of lists of nonzero integers representing a CNF formula,
      with sign indicating whether the variable is negated.

  Returns:
    satisfiable: A bool indicating whether some satisfying assignment exists.
    assignment: A dict mapping positive integers to bools, or None if no
      satisfying assignment exists.

  Examples:
    >> run_inference_dpll([[1, -2], [-1, -2]])
    >> (True, {1: True, 2: False}))

    >> run_inference_dpll([[1], [-1]])
    >> (False, None)
  """
  def __select_literal(cnf):
    for c in cnf:
        return abs(c[0])
  def get_var(cnf):
    a = set()
    for c in cnf:
      for v in c:
        a.add(v)
    return a
  def dpll(cnf, assignments={}):
    if len(cnf) == 0:
      return True, assignments
 
    if any([c is None or len(c)==0 for c in cnf]):
      return False, None
 
    l = __select_literal(cnf)
    con = True
    new_cnf = []
    for c in cnf:
      copy = c.copy()
      if l in copy:
        continue
      elif -l in copy:
        copy.remove(-l) 
        if len(copy) == 0:
          sat = False
          con = False
          break
        else:
          new_cnf.append(copy)
      else:
        new_cnf.append(c)
    if con:
      print(new_cnf)
      sat, vals = dpll(new_cnf, {**assignments, **{l: True}})
    if sat:
        return sat, vals
    con = True
    new_cnf = []
    for c in cnf:
      copy=c.copy()
      if -l in copy:
        continue
      elif l in c:
        copy.remove(l)
        if len(copy) == 0:
          sat = False
          con = False
          break
        else:
          new_cnf.append(copy)
      else:
        new_cnf.append(c)
    if con:
      sat, vals = dpll(new_cnf, {**assignments, **{l: False}})
    if sat:
      return sat, vals
 
    return False, None
  re = dpll(cnf_formula,{})
  va =get_var(cnf_formula)
  if (re[0] and len(re[1])<len(va)):
    for vv in va:
      if not vv in re[1].keys():
        re[1][vv] = True
  return(re)

Tests

In [ ]:
#assert run_inference_dpll([]) == (True, {})
#assert run_inference_dpll([[1]]) == (True, {1: True})
assert run_inference_dpll([[-1]]) == (True, {1: False})
assert run_inference_dpll([[1, 2]]) in [(True, {1: True, 2: True}), (True, {1: True, 2: False}), (True, {1: False, 2: True})]
assert run_inference_dpll([[-1, 2]]) in [(True, {1: True, 2: True}), (True, {1: False, 2: True})]
assert run_inference_dpll([[1], [-1]]) == (False, None)
assert run_inference_dpll([[1, 2, 3], [-1, -2, -3], [1, -2, 3], [-1], [-3]]) == (False, None)
assert run_inference_dpll([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]) == (True, {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True, 10: True, 11: True, 12: True, 13: True, 14: True, 15: True, 16: True, 17: True, 18: True, 19: True, 20: True, 21: True, 22: True, 23: True, 24: True, 25: True, 26: True, 27: True, 28: True, 29: True, 30: True, 31: True, 32: True})
print('Tests passed.')

(True, {1: False})
{-1}
[]
(True, {1: True})
{1, 2}
[[2]]
[]
(True, {1: True, 2: True})
{2, -1}
(False, None)
{1, -1}
[[3], [-3]]
(False, None)
{1, 2, 3, -2, -3, -1}
[[2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
[[6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [